In [ ]:
from moviepy import VideoFileClip

def count_words(text):
    return len(text.strip().split())

def calculate_wpm(video_path, transcript_text):
    clip = VideoFileClip(video_path)
    duration_minutes = clip.duration / 60.0

    total_words = count_words(transcript_text)

    if duration_minutes == 0:
        return 0.0

    wpm = total_words / duration_minutes
    return round(wpm, 2)

In [1]:
import pandas as pd

data = pd.read_csv("D:/course_project/data/datasets/dataset.csv")
data.drop(columns=["confidence_assessment"], inplace=True)

In [ ]:
import os

fragments_path = "D:/course_project/data/fragments"
subtitle_path = os.path.join(fragments_path, "subtitles")
videos_path = os.path.join(fragments_path, "videos")

for video in os.listdir(videos_path):
    video_path = os.path.join(videos_path, video)
    for fragment in os.listdir(subtitle_path):
        if video.split(".")[0] == fragment.split(".")[0]:
            video_path = os.path.join(videos_path, video)
            fragment_path = os.path.join(subtitle_path, fragment)
            break
    with open("your_transcript.txt", "r", encoding="utf-8") as f:
        transcript = f.read()
    
    wpm = calculate_wpm(video_path, transcript)
    print(f"Words per minute (WPM): {wpm}")

In [19]:
import os
from pathlib import Path
import pandas as pd
from moviepy import VideoFileClip

def count_words(text):
    return len(text.strip().split())

def calculate_wpm(video_path, subtitle_path):
    clip = VideoFileClip(str(video_path))
    duration_min = clip.duration / 60.0
    with open(subtitle_path, 'r', encoding='utf-8') as f:
        text = f.read()
    total_words = count_words(text)
    return round(total_words / duration_min, 2) if duration_min > 0 else 0.0


def collect_wpm(data_dir):
    rows = []

    videos_dir = Path(data_dir) / "videos"
    subtitles_dir = Path(data_dir) / "subtitles"

    for video_id_dir in videos_dir.iterdir():
        if not video_id_dir.is_dir():
            continue

        subtitle_id_dir = subtitles_dir / video_id_dir.name


        for video_path in video_id_dir.glob("*.mp4"):
            fragment_name = video_path.stem  # e.g., seg000
            segment_name = f"{fragment_name}"  # e.g., 8Dv2Hdf5TRg_seg000

            subtitle_path = subtitle_id_dir / f"{fragment_name}.txt"

            wpm = calculate_wpm(video_path, subtitle_path)
            rows.append({
                "segment_name": segment_name,
                "wpm": wpm
            })

    return pd.DataFrame(rows)

df_wpm = collect_wpm("D:/course_project/data/fragments")

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 784, 'fps': 24.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 130, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 30.56, 'bitrate': 921, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': 784, 'video_fps': 24.0, 'd

In [4]:
df_wpm["segment_name"] = df_wpm["segment_name"].str.replace(r'^.*?_', '', regex=True)

,segment_name,wpm
0,8Dv2Hdf5TRg_seg000,188.48
1,8Dv2Hdf5TRg_seg001,229.01
2,8Dv2Hdf5TRg_seg002,214.69
3,8Dv2Hdf5TRg_seg003,207.82
4,8Dv2Hdf5TRg_seg004,165.28
...,...,...
508,XZBZ5aBEUrs_seg018,137.30
509,XZBZ5aBEUrs_seg019,154.55
510,XZBZ5aBEUrs_seg020,280.39
511,XZBZ5aBEUrs_seg021,241.98


In [21]:
df_wpm.to_csv("D:/course_project/data/wpm.csv", index=False)

In [22]:
df_wpm['segment_name'].nunique()

513

In [10]:
df_wpm['segment_name'].isin(['Ks-_Mh1QhMc', 'P_6vDLq64gE']).sum()

0